## Synthesizing data with time-sensitive location data

In [1]:
import sys; sys.path.insert(0,'..')
from geotaste import *

In [2]:
choices = {
    'members':get_member_choices(),
    'books':get_book_choices(),
    'authors':get_author_choices(),
    'events':get_event_choices()
}

results = {
    'members':widgets.HTML('--'),
    'books':widgets.HTML('--'),
    'authors':widgets.HTML('--'),
    'events':widgets.HTML('--'),
}

parsers = {
    'members':parse_member_choices,
    'books':parse_book_choices,
    'authors':parse_author_choices,
    'events':parse_event_choices
}

data = {}

In [3]:
def get_choice_vbox(key):
    return widgets.VBox([widgets.HTML(f'<h3>{key.title()}</h3>'), results[key]] + list(choices[key].values()))

In [4]:
def parse_all_choices(choices):
    for ctype in choices:
        ctype_choices = choices[ctype]
        cdf, cdesc = parsers[ctype](ctype_choices)
        cdesc_str = ' && '.join(f'{k} in {v}' if type(v)==tuple else f'{k} is {v}' for k,v in cdesc.items())
        results[ctype].value=f'<i>{len(cdf)} rows</i><br/><b><font color="red">{cdesc_str if cdesc_str else "&nbsp;"}</font></b>'
        data[ctype] = cdf


def show_all_choices():
    b=widgets.Button(description='Filter Data')
    b.on_click(lambda x: parse_all_choices(choices))

    bcol = widgets.VBox([widgets.HTML(f'<h3>Run</h3>'), b])

    obj = widgets.HBox([
        get_choice_vbox('members'),
        get_choice_vbox('books'),
        get_choice_vbox('authors'),
        get_choice_vbox('events'),
        bcol
    ])
    return obj

In [5]:
show_all_choices()

### Logic for mapping/reporting combined data

...

In [6]:
# data['events']

In [7]:
def get_combined_filtered_events_df():
    # really this is about events?
    df = data['events']

    # make sure in member ids
    ok_members = set(data['members'].index)

    # make sure books
    df_books = data['books']
    ok_authors = set(data['authors']['author_sort_name'])
    def is_ok_author(x): return bool({y.strip() for y in x.split(';') if y.strip()} & ok_authors)
    if ok_authors:
        df_books=df_books[df_books.author.apply(is_ok_author)]
    ok_books = set(df_books.index)

    df = df[(df.member_id.isin(ok_members)) & (df.book_id.isin(ok_books))]
    
    return find_dwellings_for_member_events(df)

In [8]:
df = get_combined_filtered_events_df()
df

KeyError: 'events'

In [ ]:
# draw_choropleth(df)

In [ ]:
def get_arrond_counts(df,key='arrond_id'):
    df=df.sample(frac=1)
    arrond_counts = {n:0 for n in sorted(get_all_arrond_ids(), key=lambda x: int(x))}
    df[key] = [x.iloc[0] if type(x)==pd.Series else x for x in df[key]]
    for k,v in dict(df[key].value_counts()).items(): arrond_counts[k]=v    
    arrond_df = pd.DataFrame([arrond_counts]).T.reset_index()
    arrond_df.columns=[key, 'count']
    arrond_df['perc']=arrond_df['count'] / sum(arrond_df['count']) * 100
    return arrond_df.sort_values('perc',ascending=False)#.set_index('arrond_id')



In [ ]:
def draw_choro(df1,**kwargs):
    opts={**dict(zoom_start=12, zoom=True, heatmap=True, value='perc'), **kwargs}
    m1=draw_choropleth(df1, **opts)
    cdf1=get_arrond_counts(df1).set_index('arrond_id')
    
    htmlstr= f'''
        <div style="clear:both";>
            <br/>
            <hr/>
            <h3>Comparison table and contrast map</h3>
            <i>Below is a table of counts and percentages creating the two maps, (L)eft and (R)ight, above. 
            <br/>It also shows the (L-R) values: negative here means (R) outweighed left; positive means (L) outweighed (R).
            <br/>In the map below and right, red means that (R) outweighed (L); blue means (L) outweighed (R).
            </i>
        </div>
        <div style="clear:both";>
            {get_iframe(m1,return_str=True, height=400, width=600, float="right")}
            <div style="width:400px;">{round(cdf1,1).to_html()}<br/></div>
        </div>'
    '''
    return HTML(htmlstr)

In [ ]:
draw_choro(df)

In [9]:
display(results['members'])

HTML(value='--')